In [ ]:
import requests
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing as mp
import os
from datetime import datetime
import numpy as np

# """____for input and output path____"""

global root_path
global input_path
global output_path
global output_file_name
global url_count
global url_processed
global url_failed
global f_data


# root_path = os.path.dirname(os.path.abspath(__file__))
root_path = os.getcwd()
input_path = os.path.join(root_path, "input")
output_path = os.path.join(root_path, "output")
input_file = 'cookies.csv'
output_file_name = 'cookies_output_'

f_data = pd.DataFrame(columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
try:
    list_urls = pd.read_csv(os.path.join(input_path, input_file),
                                        delimiter="\t")['url'].values
except:
    list_urls = pd.read_csv(os.path.join(input_path, input_file),
                                    delimiter="\t")['URL'].values
url_count = len(list_urls)
print(url_count)
url_processed = 0
url_failed = 0
#____initializing date & time for output file name
today = datetime.now().strftime('%m%d_%H%M')

def get_status_code(url):
    global root_path
    global input_path
    global output_path
    global output_file_name
    global url_count
    global url_processed
    global url_failed
    global f_data
    url_count-=1
    url_processed +=1
        
    try:
        r = requests.get(url)
        print("Processing " + str(url))

        if len(r.history) > 0:
            chain = ""
            code = r.history[0].status_code
            final_url = r.url
            for resp in r.history:
                chain += resp.url + " | "
            print(str(url_processed)+" URLs Processed | "+str(url_count)+" Remaining | "+str(url_failed)+" Failed")
            df = pd.DataFrame([[url,str(code),str(len(r.history)),chain,final_url]], columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
            f_data = f_data.append(df)
            f_data.to_csv(os.path.join(output_path, output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)
            return [url,str(code),str(len(r.history)),chain,final_url]
        
        else:
            df = pd.DataFrame([[url,str(r.status_code),'n/a','n/a','n/a']], columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
            f_data = f_data.append(df)
            f_data.to_csv(os.path.join(output_path, output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)
            return [url,str(r.status_code),'n/a','n/a','n/a']
        
    except requests.ConnectionError:
        print("Error: failed to connect.")
        url_failed +=1
        print(str(url_processed)+" URLs Processed | "+str(url_count)+" Remaining | "+str(url_failed)+" Failed")
        df = pd.DataFrame([[url,'0','n/a','n/a','n/a']], columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
        f_data = f_data.append(df)
        f_data.to_csv(os.path.join(output_path, output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)
        return [url,'0','n/a','n/a','n/a']


def final_redirect(urls):
    data = []
    for i in urls:
        efr = get_status_code(i)
        data.append(efr)
                
    return data
 
    
#____Preparing arguments for multiprocessing
cpu_count = 4
print(cpu_count)
pool = ThreadPool(cpu_count)
list_split = np.array_split(list_urls, cpu_count)
general_data = pool.map(final_redirect, list_split)


#____merging list 
data_combined = []
for i in general_data:
    for j in i:
        data_combined.append(j)
        
#____transforming list data to dataframe

dataf = pd.DataFrame(data_combined,columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
                         
#___writing csv file "tab separated"
dataf.to_csv(os.path.join(output_path, "final_"+output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)




21937
4
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/20449/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vZW5hbWVsd2FyZS1jb2xhbmRlci1zZXQuaHRtbA
4 URLs Processed | 21933 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/22293/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMjIyOTMvcy9idW5ueS1iaXJkLWZsb3VyLXNhY2stdG93ZWwvY2F0ZWdvcnkvNTQv/
5 URLs Processed | 21932 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/3861/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMzg2MS9zL2hhcHB5LWhvdXItc2lnbi9jYXRlZ29yeS8xMDI4Lw
6 URLs Processed | 21931 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/wishlist/index/add/product/19271/
7 URLs Processed | 21930 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/22735/uenc/aHR0cHM6Ly93d3c

Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/14464/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMTQ0NjQvcy9iaXJkaG91c2UtZmxvd2Vycy1jaGFpci1wYWQvY2F0ZWdvcnkvMTE1Ni8
40 URLs Processed | 21897 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/2368/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vaG9tZS1kZWNvci9jYW5kbGVzLXNjZW50cy1ob21lLWRlY29yL3dpY2stdHJpbW1lci5odG1s/
41 URLs Processed | 21896 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/17689/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMTc2ODkvcy93eW50ZXItc25vd21hbi1wcmludC9jYXRlZ29yeS8xMjI0Lw
42 URLs Processed | 21895 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/2474/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vc2FpbG9ycy1rbm90LWRvb3JzdG9wLmh0bWw
43 URLs Processed | 

Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/13029/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMTMwMjkvcy9kb2dzLWNhbXBpbmctdHJpcC1pbmRvb3Itb3V0ZG9vci1ydWcvY2F0ZWdvcnkvMzIv/
75 URLs Processed | 21862 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/19607/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vZmxvcmFsLWRheWJlZC1xdWlsdC1zZXQuaHRtbA
76 URLs Processed | 21861 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/17222/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vZnVybml0dXJlL2FjY2VudC10YWJsZXMtZnVybml0dXJlL2NvY2t0YWlsLWNvZmZlZS10YWJsZXMvYnJvb2tzLWxpZnQtdG9wLWNvY2t0YWlsLXRhYmxlLmh0bWw
77 URLs Processed | 21860 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/18506/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vc2Vhc29uYWwtZGVjb3IvbWVtb3JpYWwtZGF5LWp1bHktNH

Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/20295/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMjAyOTUvcy9zZWFzaGVsbC1uYXBraW4tcmluZy1zZXQvY2F0ZWdvcnkvMzIv/
112 URLs Processed | 21825 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/2325/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20va2l0Y2hlbi1hY2Nlc3Nvcmllcy10b29scy9wbGFjZW1hdHMtdGFibGV0b3AtaG9tZS1kZWNvci9iaXJkLWxhY2UtdGFibGUtcnVubmVyLmh0bWw
113 URLs Processed | 21824 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/11915/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vaG9tZS1kZWNvci9zdG9yYWdlLWhvbWUtZGVjb3Ivc2V0LW9mLWxhcmdlLXdpcmUtYmFza2V0cy5odG1s/
114 URLs Processed | 21823 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/18524/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcva

Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/14660/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vd2FsbC1kZWNvci9yYWNrcy1zaGVsdmVzLXdhbGwtZGVjb3Ivc21hbGwtb2ZmaWNlLW9yZ2FuaXplci5odG1s/
147 URLs Processed | 21790 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/wishlist/index/add/product/21028/
148 URLs Processed | 21789 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/15440/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vZnJhbmtsaW4tY29hdC1yYWNrLXdpdGgtc2hlbHZlcy5odG1s/
149 URLs Processed | 21788 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/18662/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMTg2NjIvcy93YWxsLW1vdW50LW1ldGFsLWJyYWNrZXQvY2F0ZWdvcnkvOTQv/
150 URLs Processed | 21787 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/21189/uenc/aHR0

Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/21112/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vc2Vhc29uYWwtZGVjb3IvZmFsbC1oYXJ2ZXN0LWRlY29yYXRpbmcvZmFsbC1mbG93ZXItdmFzZS13YWxsLWhhbmdpbmcuaHRtbA
183 URLs Processed | 21754 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/18658/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMTg2NTgvcy9zZWFzaWRlLWxhY2UtY3VydGFpbi1jb2xsZWN0aW9uL2NhdGVnb3J5LzExNTgv/
184 URLs Processed | 21753 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/wishlist/index/add/product/16347/
185 URLs Processed | 21752 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/wishlist/index/add/product/22535/
186 URLs Processed | 21751 Remaining | 0 Failed
Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/5311/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvNTMxMS9zL2N

Processing https://www.sturbridgeyankee.com/catalog/product_compare/add/product/2102/uenc/aHR0cHM6Ly93d3cuc3R1cmJyaWRnZXlhbmtlZS5jb20vY2F0YWxvZy9wcm9kdWN0L3ZpZXcvaWQvMjEwMi9zL2F1dHVtbi1mcmFtZS9jYXRlZ29yeS8xMDA0Lw
219 URLs Processed | 21718 Remaining | 0 Failed


In [86]:
data_combined = []
for i in general_data:
    for j in i:
        data_combined.append(j)
